In [1]:
import pandas as pd
import xarray as xr
import tensorflow as tf

from datetime import datetime
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras.layers import Cropping2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, GlobalMaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout
from tensorflow.keras.layers import AveragePooling2D, Flatten, Add
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/homes/rjackson/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
label_df = pd.read_csv('../notebooks/lidar_labels.csv')

date_list = np.array([datetime.datetime.strptime(x, '%Y-%m-%d').date() for x in label_df["Date"].values])
start_time_list = np.array([datetime.datetime.strptime(x[0:4], '%H%M').time() for x in label_df["Time"].values])
end_time_list = np.array([datetime.datetime.strptime(x[5:], '%H%M').time() for x in label_df["Time"].values])


def get_label(dt):
    label_ind = np.where(np.logical_and.reduce(
        (date_list == dt.date(), start_time_list <= dt.time(), end_time_list > dt.time())))
    if not label_ind[0].size:
        return -1
    my_strings = np.array(
            [x.lower() for x in label_df["Label"].values[label_ind]])
    num_cloud = len(np.where(my_strings == "cloudy")[0]) + \
            len(np.where(my_strings == "cloud")[0])
    num_clear = len(np.where(my_strings == "clear")[0])
    num_rain = len(np.where(my_strings == "rain")[0])
    my_string = label_df["Label"].values[label_ind][0]

    pct_cloud = num_cloud/len(my_strings)

    if num_rain > 0:
        return 2
    elif pct_cloud < 0.25:
        return 0
    elif pct_cloud >= 0.25 and pct_cloud <= 0.75:
        return 1
    else:
        return 1

    raise ValueError("Invalid value %s for label" % my_string)


def nn_classifier(input_array, num_layers=2, num_channel_start=2):
    inp = Input(shape=(input_array.shape[1],), name='input')
    layer = inp
    num_channels = num_channel_start
    for i in range(num_layers):
        
        layer = Dense(num_channels)(layer)
        #layer = Dropout(0.25)(layer)
        layer = BatchNormalization()(layer)
        layer = Activation('relu')(layer)
        #layer = Dropout(0.25)(layer)
        #layer = Dropout(0.75)(layer)
        #num_channels = num_channels * 2
        
    output = Dense(3, activation='softmax',
            name='output')(layer)

    return Model(inp, output)


def dt64_to_dt(dt):
    ts = (dt - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    return datetime.datetime.utcfromtimestamp(ts)

my_ds = xr.open_mfdataset('/lambda_stor/data/rjackson/coverage_product/*.nc')

/homes/rjackson/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/ipykernel_launcher.py:59: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

/homes/rjackson/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


In [10]:
labels = my_ds['time'].values
labels = np.array([get_label(dt64_to_dt(x)) for x in labels])
print(my_ds.variables.keys()) 
time = my_ds['time'].values
range_bins = my_ds['range_bins'].values
feature_list = ['snrgt1.000000', 'snrgt3.000000', 'snrgt5.000000']


x = np.squeeze(np.concatenate([my_ds[x].values for x in feature_list], axis=1))
labels = np.squeeze(labels)
feature_labels = []
for feat in feature_list:
    for i in range(len(my_ds.range_bins.values)):
        feature_labels.append('%s%d' % (feat, my_ds.range_bins[i]))

/homes/rjackson/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/ipykernel_launcher.py:56: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


KeysView(Frozen({'range_bins': <xarray.IndexVariable 'range_bins' (range_bins: 99)>
array([   60.,   180.,   300.,   420.,   540.,   660.,   780.,   900.,  1020.,
        1140.,  1260.,  1380.,  1500.,  1620.,  1740.,  1860.,  1980.,  2100.,
        2220.,  2340.,  2460.,  2580.,  2700.,  2820.,  2940.,  3060.,  3180.,
        3300.,  3420.,  3540.,  3660.,  3780.,  3900.,  4020.,  4140.,  4260.,
        4380.,  4500.,  4620.,  4740.,  4860.,  4980.,  5100.,  5220.,  5340.,
        5460.,  5580.,  5700.,  5820.,  5940.,  6060.,  6180.,  6300.,  6420.,
        6540.,  6660.,  6780.,  6900.,  7020.,  7140.,  7260.,  7380.,  7500.,
        7620.,  7740.,  7860.,  7980.,  8100.,  8220.,  8340.,  8460.,  8580.,
        8700.,  8820.,  8940.,  9060.,  9180.,  9300.,  9420.,  9540.,  9660.,
        9780.,  9900., 10020., 10140., 10260., 10380., 10500., 10620., 10740.,
       10860., 10980., 11100., 11220., 11340., 11460., 11580., 11700., 11820.]), 'time': <xarray.IndexVariable 'time' (time: 1

In [5]:
def train_nn(num_layers, num_channels, x_test, x_train, y_test, y_train):
    model = nn_classifier(x_train, num_layers=num_layers, num_channel_start=num_channels)
    model.summary()
    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    checkpointer = ModelCheckpoint(
        filepath=('/homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-%dlayers-%dnodes-{epoch:03d}.hdf5' % (num_layers, num_channels)),
        verbose=1)
    early_stop = EarlyStopping(patience=500)
    return model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3000, callbacks=[checkpointer, early_stop], initial_epoch=epoch_no)

In [7]:
def rus(x, y, class_no, chance=0.33):
    x_new = []
    y_new = []
    rands = np.random.random(len(y))
    new_inds = np.argwhere(np.logical_or(y != class_no, rands < chance))
    x_new = x[new_inds, :]
    y_new = y[new_inds]
    return np.squeeze(x_new), np.squeeze(y_new)

(8151, 1, 297)

In [11]:
n_layers = np.arange(2, 12, 1)
n_nodes = 2**(np.arange(6, 15, 1))
x = np.where(np.isfinite(x), x, 0)
x = scale(np.squeeze(x))
labels = np.squeeze(labels)
x, labels = rus(x, labels, 0)
x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.20)

epoch_no = 1
y_train = tf.one_hot(y_train, depth=3).numpy()
y_test = tf.one_hot(y_test, depth=3).numpy()
y = labels
#for i in n_layers:
#    for j in n_nodes:
history = train_nn(8, 128, x_test, x_train, y_test, y_train)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 297)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               38144     
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0     

Epoch 19/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3536 - accuracy: 0.8602 - val_loss: 0.3692 - val_accuracy: 0.8411

Epoch 00019: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-019.hdf5
Epoch 20/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3621 - accuracy: 0.8567 - val_loss: 0.3645 - val_accuracy: 0.8429

Epoch 00020: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-020.hdf5
Epoch 21/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3465 - accuracy: 0.8550 - val_loss: 0.3577 - val_accuracy: 0.7806

Epoch 00021: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-021.hdf5
Epoch 22/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3551 - accuracy: 0.8528 - val_loss: 0.3589 - val_accuracy: 0.8472

Epoch 00022: saving model to /homes/rjackson/arming_the_edge/models/dnn


Epoch 00050: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-050.hdf5
Epoch 51/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3505 - accuracy: 0.8657 - val_loss: 0.3622 - val_accuracy: 0.7842

Epoch 00051: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-051.hdf5
Epoch 52/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3416 - accuracy: 0.8684 - val_loss: 0.3639 - val_accuracy: 0.7842

Epoch 00052: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-052.hdf5
Epoch 53/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3484 - accuracy: 0.8745 - val_loss: 0.3541 - val_accuracy: 0.8490

Epoch 00053: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-053.hdf5
Epoch 54/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3471 - accuracy: 0.873

Epoch 82/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3602 - accuracy: 0.8689 - val_loss: 0.3714 - val_accuracy: 0.8472

Epoch 00082: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-082.hdf5
Epoch 83/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3562 - accuracy: 0.8706 - val_loss: 0.3712 - val_accuracy: 0.8509

Epoch 00083: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-083.hdf5
Epoch 84/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3620 - accuracy: 0.8685 - val_loss: 0.3656 - val_accuracy: 0.8490

Epoch 00084: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-084.hdf5
Epoch 85/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3410 - accuracy: 0.8777 - val_loss: 0.3659 - val_accuracy: 0.8484

Epoch 00085: saving model to /homes/rjackson/arming_the_edge/models/dnn

205/205 [==============================] - 1s 4ms/step - loss: 0.3519 - accuracy: 0.8749 - val_loss: 0.3742 - val_accuracy: 0.8478

Epoch 00113: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-113.hdf5
Epoch 114/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3517 - accuracy: 0.8781 - val_loss: 0.3778 - val_accuracy: 0.8490

Epoch 00114: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-114.hdf5
Epoch 115/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3588 - accuracy: 0.8763 - val_loss: 0.3815 - val_accuracy: 0.8447

Epoch 00115: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-115.hdf5
Epoch 116/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3478 - accuracy: 0.8752 - val_loss: 0.3759 - val_accuracy: 0.8509

Epoch 00116: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclas

...

205/205 [==============================] - 1s 4ms/step - loss: 0.3854 - accuracy: 0.8687 - val_loss: 0.3844 - val_accuracy: 0.8545

Epoch 00392: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-392.hdf5
Epoch 393/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3846 - accuracy: 0.8669 - val_loss: 0.3853 - val_accuracy: 0.8521

Epoch 00393: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-393.hdf5
Epoch 394/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3901 - accuracy: 0.8682 - val_loss: 0.3790 - val_accuracy: 0.8539

Epoch 00394: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-394.hdf5
Epoch 395/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3836 - accuracy: 0.8752 - val_loss: 0.3854 - val_accuracy: 0.8496

Epoch 00395: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclas

205/205 [==============================] - 1s 4ms/step - loss: 0.3900 - accuracy: 0.8675 - val_loss: 0.3813 - val_accuracy: 0.8545

Epoch 00423: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-423.hdf5
Epoch 424/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3897 - accuracy: 0.8649 - val_loss: 0.3842 - val_accuracy: 0.8515

Epoch 00424: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-424.hdf5
Epoch 425/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.4030 - accuracy: 0.8553 - val_loss: 0.3802 - val_accuracy: 0.8539

Epoch 00425: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-425.hdf5
Epoch 426/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3908 - accuracy: 0.8728 - val_loss: 0.3881 - val_accuracy: 0.8527

Epoch 00426: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclas

205/205 [==============================] - 1s 4ms/step - loss: 0.4073 - accuracy: 0.8537 - val_loss: 0.3789 - val_accuracy: 0.8533

Epoch 00454: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-454.hdf5
Epoch 455/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.4017 - accuracy: 0.8620 - val_loss: 0.3801 - val_accuracy: 0.8564

Epoch 00455: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-455.hdf5
Epoch 456/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3991 - accuracy: 0.8623 - val_loss: 0.3827 - val_accuracy: 0.8545

Epoch 00456: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-456.hdf5
Epoch 457/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3844 - accuracy: 0.8658 - val_loss: 0.3843 - val_accuracy: 0.8515

Epoch 00457: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclas

205/205 [==============================] - 1s 4ms/step - loss: 0.4001 - accuracy: 0.8599 - val_loss: 0.3925 - val_accuracy: 0.8515

Epoch 00485: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-485.hdf5
Epoch 486/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.4077 - accuracy: 0.8569 - val_loss: 0.3874 - val_accuracy: 0.8484

Epoch 00486: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-486.hdf5
Epoch 487/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3784 - accuracy: 0.8714 - val_loss: 0.3893 - val_accuracy: 0.8521

Epoch 00487: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-487.hdf5
Epoch 488/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.4025 - accuracy: 0.8605 - val_loss: 0.3862 - val_accuracy: 0.8515

Epoch 00488: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclas

205/205 [==============================] - 1s 4ms/step - loss: 0.4021 - accuracy: 0.8627 - val_loss: 0.3842 - val_accuracy: 0.8533

Epoch 00516: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-516.hdf5
Epoch 517/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3904 - accuracy: 0.8643 - val_loss: 0.3855 - val_accuracy: 0.8478

Epoch 00517: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-517.hdf5
Epoch 518/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.3874 - accuracy: 0.8660 - val_loss: 0.3872 - val_accuracy: 0.8509

Epoch 00518: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-8layers-128nodes-518.hdf5
Epoch 519/3000
205/205 [==============================] - 1s 4ms/step - loss: 0.4024 - accuracy: 0.8664 - val_loss: 0.3896 - val_accuracy: 0.8496

Epoch 00519: saving model to /homes/rjackson/arming_the_edge/models/dnn_scp/nnclas

In [ ]:
model = nn_classifier(x_train, num_layers=2, num_channel_start=2048)
model.summary()
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
#checkpointer = ModelCheckpoint(
#        filepath=('/homes/rjackson/arming_the_edge/models/dnn_scp/nnclassifier-%dlayers-%dnodes-{epoch:03d}.hdf5' % (num_layers, num_channels)),
#        verbose=1)
early_stop = EarlyStopping(patience=500)
results = model.evaluate(x_train, y_train, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

In [24]:
np.log(len(np.argwhere(x_train > 0))/len(np.argwhere(x_train == 0)))

-2.575843422328888